In [77]:
import psycopg2
import shapely

#TESTER DE FAIRE AVEC ÇA : http://postgis.net/docs/ST_ApproximateMedialAxis.html

con = psycopg2.connect(host="localhost", port=54320, database="vtile", user='ubuntu')
cur = con.cursor()

print('on supprime la table de restriction des données sur la TAN')
cmd = "drop table tan;"
cur.execute(cmd)

cmd = """
select * 
into tan
from lines
where code in ('2', '3')
and operator = 'SEMITAN'
"""
cur.execute(cmd)

print('on supprime la table de travail')
cmd = "drop table tan_segmentized;"
cur.execute(cmd)

print('on crée la table de travail en éclatant les segments')
cmd = """
select id as line_id, id || '-' || (ST_Dump(geom)).path[1] as segment_id , 
code, name, colour, mode, network,
(ST_Dump(geom)).path[1], 
(ST_Dump(geom)).geom, 
st_length((ST_Dump(geom)).geom) as length 
into tan_segmentized
from tan;"""
cur.execute(cmd)
con.commit()


print("on supprime les segments de chaque ligne qui est inclus dans un buffer de 20m d'un autre segment plus long")
cmd = """
select line_id, segment_id, code, name, colour, mode, network, 
    st_astext(geom) as wkt_geom, 
    length,
    geom
from tan_segmentized
order by line_id, length desc;
"""
cur.execute(cmd)
rows = cur.fetchall()
lines = {}
import shapely
from shapely.wkt import loads
from shapely.ops import linemerge

BUFFER_DISTANCE = 20.0

for row in rows:
    line_id = row[0]

    if line_id not in lines:
        lines[line_id] = []
    lines[line_id].append({
            "segment_id": row[1],
            "wkt_geom": row[7],
            "length": row[8],
            "geom": loads(row[7])
    })

#Pour chaque ligne, on prend le premier segment (le plus long), et on teste chauque autre segment : 
# - s'il est entièrement inclus dans la ligne en cours de reconstruction => à supprimer
# - s'il n'est pas connectés => on le garde pour plus tard
def analyse_line_segments(segments_to_be_analysed, reconstructed_line, depth):
    new_segments_to_be_deleted = []
    segments_to_be_later_analysed = []
    new_shape = reconstructed_line
    if depth >= 100:
        print("on atteint l'itération 100, on arrête le traitement")
        print("nombre d'items encore à analyser : {:d}".format(len(segments_to_be_analysed)))
    else:
        buffered = new_shape.buffer(BUFFER_DISTANCE)
        for s in segments_to_be_analysed:
            if s["segment_id"] == "Line:Relation:2166523-28":
                print("Itération {:d} - Segment included in buffer : {:b}".format(depth, buffered.contains(s['geom'])))
                print("Itération {:d} - Segment intersect : {:b}".format(depth, new_shape.intersects(s['geom'])))
            if buffered.contains(s['geom']):
                new_segments_to_be_deleted.append(s)
            elif new_shape.intersects(s['geom']) :
                new_shape = new_shape.union(s['geom'])
            else: #segment non inclus et qui n'intersecte pas la ligne en cours de construction
                segments_to_be_later_analysed.append(s)
                   
        if segments_to_be_later_analysed :
            if len(segments_to_be_later_analysed) == len(segments_to_be_analysed):
                print("Problème, on n'améliore plus ! {:d} segments restants - itération {:d}".format(
                    len(segments_to_be_analysed), depth))
            else:
                result = analyse_line_segments(segments_to_be_later_analysed, 
                                               new_shape, 
                                               depth+1)
                new_segments_to_be_deleted.extend(result["to_be_deleted"])
                segments_to_be_later_analysed = result["not_categorized"]
                new_shape = result["new_line"]
    #print("--- itération {:d}-----".format(depth))
    #print("nb to be deleted : {}".format(len(new_segments_to_be_deleted)))
    #print("nb to be analysed : {}".format(len(segments_to_be_later_analysed)))
    #print("length of reconstructed line : {:f}".format(new_shape.length))
    return {"to_be_deleted" : new_segments_to_be_deleted, 
            "not_categorized": segments_to_be_later_analysed,
            "new_line": new_shape, 
            "new_buffer": new_shape.buffer(BUFFER_DISTANCE)}
con.commit()
cmd = """
alter table tan_segmentized add column segment_state varchar;
"""
cur.execute(cmd)
cmd = """
update tan_segmentized set segment_state = 'OK';
"""
cur.execute(cmd)

segments_to_be_deleted = []
segments_to_be_later_analysed = []
for line_id, l in lines.items():
    #Line:Relation:2166523-28 et Line:Relation:2166523-27
    #segment1 = [s["geom"] for s in l if s["segment_id"] == "Line:Relation:2166523-28"][0]
    #segment2 = [s["geom"] for s in l if s["segment_id"] == "Line:Relation:2166523-27"][0]
    #print(segment1.intersects(segment2))
    #new_shape = linemerge([segment1, segment2])
    #print(new_shape.intersects(segment1))

    print("traitement de la ligne : {} (nombre de segments : {:d})".format(line_id, len(l)))
    first_segment = l.pop(0)
    print("premier segment : " + first_segment["segment_id"])
    reconstructed_line = first_segment['geom']
    result = analyse_line_segments(l, reconstructed_line, 1)
    segments_to_be_deleted = result["to_be_deleted"]
    for segment in segments_to_be_deleted:
        cur.execute("update tan_segmentized set segment_state = 'deleted' where segment_id='{}'".format(segment['segment_id']))
    for segment in result["not_categorized"]:
        cur.execute("update tan_segmentized set segment_state = 'undefined' where segment_id='{}'".format(segment['segment_id']))
        
    print("  {:d} segments à supprimer".format(len(result["to_be_deleted"])))
    print("  {:d} segments avec un statut non défini".format(len(result["not_categorized"])))

cur.execute("delete from tan_segmentized where segment_state != 'OK';")    
con.commit()
con.close()

on supprime la table de restriction des données sur la TAN
on supprime la table de travail
on crée la table de travail en éclatant les segments
on supprime les segments de chaque ligne qui est inclus dans un buffer de 20m d'un autre segment plus long
traitement de la ligne : Line:Relation:2166523 (nombre de segments : 61)
premier segment : Line:Relation:2166523-26
Itération 1 - Segment included in buffer : 0
Itération 1 - Segment intersect : 1
  31 segments à supprimer
  0 segments avec un statut non défini
traitement de la ligne : Line:Relation:2166524 (nombre de segments : 151)
premier segment : Line:Relation:2166524-23
  78 segments à supprimer
  0 segments avec un statut non défini


In [78]:
con = psycopg2.connect(host="localhost", port=54320, database="vtile", user='ubuntu')
cur = con.cursor()

print("On crée la table qui contient les segments (avec le nombre de lignes)")
cmd = "drop table tan_group;"
cur.execute(cmd)
cmd = "drop table tan_detail;"
cur.execute(cmd)

cmd="""
select geom, count(distinct line_id), array_agg(code) as codes
into tan_group
from tan_segmentized
group by geom
"""
cur.execute(cmd)
cmd="""
select segment_id, geom, code, name, colour, mode, network
into tan_detail
from tan_segmentized
"""
cur.execute(cmd)
con.commit()
con.close()

On crée la table qui contient les segments (avec le nombre de lignes)


## la requete pour trex :

```
select d.*, g.count, array_position(codes, d.code) as offset_order
from tan_group g left join tan_detail d
on g.geom = d.geom
```


In [83]:
import folium

con = psycopg2.connect(host="localhost", port=54320, database="vtile", user='ubuntu')
cur = con.cursor()

cmd="""
select ST_AsGeoJSON(st_transform(geom, 4326))
from tan_detail
"""
cur.execute(cmd)
rows = cur.fetchall()
geojsons = []
for row in rows:
    geojsons.append(row[0])
    
macarte = folium.Map(location=[47.2168,-1.5512], zoom_start=13, tiles='Cartodb Positron')
for i in geojsons:
    g = folium.GeoJson(i,
                      style_function=lambda x: {'fillColor':'#ffff00'})
    macarte.add_child(g)
con.close()
macarte